# **Logistic Regression**

In [65]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

from sklearn.feature_selection import RFE, RFECV

from matplotlib.colors import ListedColormap 

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, log_loss, brier_score_loss, confusion_matrix
from sklearn.decomposition import PCA

# Display all columns and rows of the dataset
pd.options.display.max_columns = None
pd.options.display.max_rows = None
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [54]:
# Load dataset
df = pd.read_csv("/content/gdrive/MyDrive/Thesis/MODELING/Encoded.csv")

In [55]:
# Size of dataset
df.shape

(28990, 291)

In [56]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
df.head()

,tourney_id,location,draw_size,series,money_prize,year,month,day,match_num,best_of,round,A_id,B_id,A_rank,B_rank,A_hand,A_ht,A_age,B_hand,B_ht,B_age,A_rank_points,B_rank_points,A_B365,B_B365,Label,home_advantage,A_rank_log,B_rank_log,A_rank_points_log,B_rank_points_log,match_quality1,match_quality2,surface_court,B365_ratio,A_spread,B_spread,A_wins_career,A_losses_career,A_matches_career,A_titles_career,B_wins_career,B_losses_career,B_matches_career,B_titles_career,A_wins_year,A_losses_year,A_matches_year,A_titles_year,B_wins_year,B_losses_year,B_matches_year,B_titles_year,A_matches_year_per,B_matches_year_per,A_wins_semester,A_losses_semester,A_matches_semester,A_titles_semester,B_wins_semester,B_losses_semester,B_matches_semester,B_titles_semester,A_wins_recent,A_losses_recent,A_matches_recent,A_titles_recent,B_wins_recent,B_losses_recent,B_matches_recent,B_titles_recent,A_matches_per_surface,A_wins_per_surface,A_losses_per_surface,A_titles_per_surface,B_matches_per_surface,B_wins_per_surface,B_losses_per_surface,B_titles_per_surface,A_surface_per,B_surface_per,A_wins_tour,A_losses_tour,A_matches_tour,A_titles_tour,B_wins_tour,B_losses_tour,B_matches_tour,B_titles_tour,A_tour_per,B_tour_per,A_round_wins,A_round_losses,A_round_matches,A_round_per,B_round_wins,B_round_losses,B_round_matches,B_round_per,A_elo_rating,B_elo_rating,A_ratio_ace/df_year,B_ratio_ace/df_year,A_ace_career,A_df_career,A_svpt_career,A_1stIn_career,A_1stWon_career,A_2ndWon_career,A_SvGms_career,A_bpSaved_career,A_bpFaced_career,A_2ndIn_career,A_1stSvrptWon_career,A_2ndSvrptWon_career,A_1stServeWon_career,A_2ndServeWon_career,A_1stServe_career,A_bpSavedratio_career,A_bpWon_career,A_bpConv_career,B_ace_career,B_df_career,B_svpt_career,B_1stIn_career,B_1stWon_career,B_2ndWon_career,B_SvGms_career,B_bpSaved_career,B_bpFaced_career,B_2ndIn_career,B_1stSvrptWon_career,B_2ndSvrptWon_career,B_1stServeWon_career,B_2ndServeWon_career,B_1stServe_career,B_bpSavedratio_career,B_bpWon_career,B_bpConv_career,A_ace_year,A_df_year,A_svpt_year,A_1stIn_year,A_1stWon_year,A_2ndWon_year,A_SvGms_year,A_bpSaved_year,A_bpFaced_year,A_bpWon_year,A_bpConv_year,A_2ndIn_year,A_1stSvrptWon_year,A_2ndSvrptWon_year,A_1stServeWon_year,A_2ndServeWon_year,A_1stServe_year,A_bpSavedratio_year,B_ace_year,B_df_year,B_svpt_year,B_1stIn_year,B_1stWon_year,B_2ndWon_year,B_SvGms_year,B_bpSaved_year,B_bpFaced_year,B_bpWon_year,B_bpConv_year,B_2ndIn_year,B_1stSvrptWon_year,B_2ndSvrptWon_year,B_1stServeWon_year,B_2ndServeWon_year,B_1stServe_year,B_bpSavedratio_year,A_over_B,B_over_A,A_Win_oppon,B_Win_oppon,A_wstreak,A_lstreak,A_days_inactive,B_wstreak,B_lstreak,B_days_inactive,A_momentum,B_momentum,preferred_hand,rank,rank_points,rank_log,rank_points_log,age,height,spread,B365,wins_career,losses_career,matches_career,titles_career,wins_year,losses_year,matches_year,titles_year,matches_year_per,wins_semester,losses_semester,matches_semester,titles_semester,wins_recent,losses_recent,matches_recent,titles_recent,matches_per_surface,wins_per_surface,losses_per_surface,titles_per_surface,surface_per,wins_tour,losses_tour,matches_tour,titles_tour,tour_per,round_wins,round_losses,round_matches,round_per,elo_rating,ratio_ace/df_year,ace_career,df_career,svpt_career,1stIn_career,1stWon_career,2ndWon_career,SvGms_career,bpSaved_career,bpFaced_career,bpWon_career,bpConv_career,2ndIn_career,1stSvrptWon_career,2ndSvrptWon_career,1stServeWon_career,2ndServeWon_career,1stServe_career,bpSavedratio_career,ace_year,df_year,svpt_year,1stIn_year,1stWon_year,2ndWon_year,SvGms_year,bpSaved_year,bpFaced_year,bpWon_year,bpConv_year,2ndIn_year,1stSvrptWon_year,2ndSvrptWon_year,1stServeWon_year,2ndServeWon_year,1stServe_year,bpSavedratio_year,headtohead,Win_oppon,wstreak,lstreak,days_inactive,momentum,ace_common_op,df_common_op,svpt_common_op,1stIn_common_op,1stWon_common_op,2ndWon_common_op,SvGms_common_op,bpSaved_common_op,bpFaced_common_op,2ndIn_common_op,bpWon_common_op,bpConv_common_op,1stSvrptWon_common_op,2ndSvrptWon_c

In [57]:
# keep only differences
df.drop(columns=['A_rank', 'B_rank', 'A_hand', 'B_hand', 'A_ht', 'B_ht', 'A_age', 'B_age', 'A_rank_points', 'B_rank_points', 'A_B365', 'B_B365', 'A_rank_log', 'B_rank_log',
                 'A_rank_points_log', 'B_rank_points_log', 'A_spread', 'B_spread', 'A_wins_career', 'A_losses_career', 'A_matches_career', 'A_titles_career', 'B_wins_career', 'B_losses_career',
                 'B_matches_career', 'B_titles_career', 'A_wins_year', 'A_losses_year', 'A_matches_year', 'A_titles_year', 'B_wins_year', 'B_losses_year', 'B_matches_year', 'B_titles_year', 
                 'A_matches_year_per', 'B_matches_year_per', 'A_wins_semester', 'A_losses_semester',  'A_matches_semester', 'A_titles_semester', 'B_wins_semester', 'B_losses_semester',
                 'B_matches_semester', 'B_titles_semester', 'A_wins_recent', 'A_losses_recent', 'A_matches_recent', 'A_titles_recent', 'B_wins_recent', 'B_losses_recent', 'B_matches_recent', 'B_titles_recent', 
                 'A_matches_per_surface', 'A_wins_per_surface', 'A_losses_per_surface', 'A_titles_per_surface', 'B_matches_per_surface', 'B_wins_per_surface', 'B_losses_per_surface', 'B_titles_per_surface',
                 'A_surface_per', 'B_surface_per', 'A_wins_tour', 'A_losses_tour', 'A_matches_tour', 'A_titles_tour', 'B_wins_tour', 'B_losses_tour', 'B_matches_tour', 'B_titles_tour', 'A_tour_per', 'B_tour_per',
                 'A_round_wins', 'A_round_losses', 'A_round_matches', 'A_round_per', 'B_round_wins', 'B_round_losses', 'B_round_matches', 'B_round_per', 'A_elo_rating', 'B_elo_rating', 'A_ratio_ace/df_year', 'B_ratio_ace/df_year',
                 'A_ace_career', 'A_df_career', 'A_svpt_career', 'A_1stIn_career', 'A_1stWon_career', 'A_2ndWon_career', 'A_SvGms_career', 'A_bpSaved_career', 'A_bpFaced_career', 'A_2ndIn_career', 'A_1stSvrptWon_career', 
                 'A_2ndSvrptWon_career', 'A_1stServeWon_career', 'A_2ndServeWon_career', 'A_1stServe_career', 'A_bpSavedratio_career', 'A_bpWon_career', 'A_bpConv_career', 'B_ace_career', 'B_df_career',
                 'B_svpt_career', 'B_1stIn_career', 'B_1stWon_career', 'B_2ndWon_career', 'B_SvGms_career', 'B_bpSaved_career', 'B_bpFaced_career', 'B_2ndIn_career', 'B_1stSvrptWon_career', 'B_2ndSvrptWon_career',
                 'B_1stServeWon_career', 'B_2ndServeWon_career', 'B_1stServe_career', 'B_bpSavedratio_career', 'B_bpWon_career', 'B_bpConv_career', 'A_ace_year', 'A_df_year', 'A_svpt_year', 'A_1stIn_year', 'A_1stWon_year',
                 'A_2ndWon_year', 'A_SvGms_year', 'A_bpSaved_year', 'A_bpFaced_year', 'A_bpWon_year', 'A_bpConv_year', 'A_2ndIn_year', 'A_1stSvrptWon_year', 'A_2ndSvrptWon_year', 'A_1stServeWon_year', 'A_2ndServeWon_year',
                 'A_1stServe_year', 'A_bpSavedratio_year', 'B_ace_year', 'B_df_year', 'B_svpt_year', 'B_1stIn_year', 'B_1stWon_year', 'B_2ndWon_year', 'B_SvGms_year', 'B_bpSaved_year', 'B_bpFaced_year', 'B_bpWon_year',
                 'B_bpConv_year', 'B_2ndIn_year', 'B_1stSvrptWon_year', 'B_2ndSvrptWon_year', 'B_1stServeWon_year', 'B_2ndServeWon_year', 'B_1stServe_year', 'B_bpSavedratio_year', 'A_over_B', 'B_over_A', 'A_Win_oppon', 
                 'B_Win_oppon', 'A_wstreak', 'A_lstreak', 'A_days_inactive', 'B_wstreak', 'B_lstreak', 'B_days_inactive', 'A_momentum', 'B_momentum'], 
        inplace=True)

In [59]:
def training(MODEL, X, y, SCALER, PCAer, n=20):
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
  
  if SCALER == True:
    print('USING SCALING..')
    scaler = StandardScaler()
    #scaler = MinMaxScaler()  
    #scaler = MaxAbsScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
  
  if PCAer == True:
    print('USING PCA..')
    pca_tf = PCA(n_components=n)
    pca_tf.fit(X_train)
    X_train = pca_tf.transform(X_train)
    X_test = pca_tf.transform(X_test)

  model = MODEL
  start_time = datetime.datetime.now()
  model.fit(X_train, y_train)
  end_time = datetime.datetime.now()
  elapsed_time = end_time - start_time
  print('Training time:', str(elapsed_time))
  print()

  predict = model.predict(X_test)
  print("\nAccuracy               :", round(accuracy_score(y_test, predict) * 100, 3), '%.')
  print("Precision              :", round(precision_score(y_test, predict) * 100, 3), '%.')
  print("Recall Score           :", round(recall_score(y_test, predict) * 100, 3), '%.')
  print("F1 Score               :", round(f1_score(y_test, predict) * 100, 3), '%.')
  print("Log-Loss               :", round(log_loss(y_test, predict), 3), '.')
  print("Brier Score            :", round(brier_score_loss(y_test, predict) * 100, 3), '%.')
  print("Confusion Matrix")
  print(confusion_matrix(y_test, predict))

  print('\nTrain Score            :', round(model.score(X_train, y_train) * 100,3), '%.')
  print('Test Score             :', round(model.score(X_test, y_test) * 100, 3), '%.')

In [60]:
df.sample(10)

,tourney_id,location,draw_size,series,money_prize,year,month,day,match_num,best_of,round,A_id,B_id,Label,home_advantage,match_quality1,match_quality2,surface_court,B365_ratio,preferred_hand,rank,rank_points,rank_log,rank_points_log,age,height,spread,B365,wins_career,losses_career,matches_career,titles_career,wins_year,losses_year,matches_year,titles_year,matches_year_per,wins_semester,losses_semester,matches_semester,titles_semester,wins_recent,losses_recent,matches_recent,titles_recent,matches_per_surface,wins_per_surface,losses_per_surface,titles_per_surface,surface_per,wins_tour,losses_tour,matches_tour,titles_tour,tour_per,round_wins,round_losses,round_matches,round_per,elo_rating,ratio_ace/df_year,ace_career,df_career,svpt_career,1stIn_career,1stWon_career,2ndWon_career,SvGms_career,bpSaved_career,bpFaced_career,bpWon_career,bpConv_career,2ndIn_career,1stSvrptWon_career,2ndSvrptWon_career,1stServeWon_career,2ndServeWon_career,1stServe_career,bpSavedratio_career,ace_year,df_year,svpt_year,1stIn_year,1stWon_year,2ndWon_year,SvGms_year,bpSaved_year,bpFaced_year,bpWon_year,bpConv_year,2ndIn_year,1stSvrptWon_year,2ndSvrptWon_year,1stServeWon_year,2ndServeWon_year,1stServe_year,bpSavedratio_year,headtohead,Win_oppon,wstreak,lstreak,days_inactive,momentum,ace_common_op,df_common_op,svpt_common_op,1stIn_common_op,1stWon_common_op,2ndWon_common_op,SvGms_common_op,bpSaved_common_op,bpFaced_common_op,2ndIn_common_op,bpWon_common_op,bpConv_common_op,1stSvrptWon_common_op,2ndSvrptWon_common_op,1stServeWon_common_op,2ndServeWon_common_op,1stServe_common_op,bpSavedratio_common_op
10899,403,39.0,96,3.0,4720380,2014,3,19,64,3,1.0,104871,104925,0,0,50.000000,6.584963,4.0,0.060000,3.0,46.000000,-9945.000000,4.584963,-3.512684,0.271047,0.000000,30.332091,15.98,-177.0,54.0,-123.0,-26.0,-11.0,-1.0,-12.0,0.0,-0.011111,59.0,41.0,100.0,1.0,99.0,65.0,164.0,5.0,-98.0,-120.0,22.0,-18.0,-0.416623,-11.0,2.0,-9.0,-26.0,-44.642857,-32.0,27.0,-5.0,-46.962233,-549.86,-212.097943,2.604361,1.998768,1.685364,-4.374638,-3.213490,1.069806,-0.437051,0.956498,1.702132,-1.911872,-0.128857,6.060002,-3.564877,-3.635005,-0.012703,-0.083548,-0.067657,0.014080,1.981649,2.901525,5.004394,-2.411734,-1.905660,1.605841,-0.185319,1.224347,2.007495,-1.766606,-0.087052,7.416128,-3.417679,-3.402688,-0.015035,-0.099309,-0.068327,0.064210,-4.0,-1.0,-5.0,0.0,-13.0,-0.1185,2.552592,2.081637,1.610683,-4.648423,-3.252519,0.913983,-0.382764,0.993578,1.826385,6.259106,-1.814595,-0.121399,-3.307614,-3.695745,-0.014231,-0.091427,-0.066830,0.006606
15326,580,1.0,128,2.0,19703000,2016,1,18,103,5,0.0,104559,104312,0,0,73.000000,10.317413,4.0,0.731818,3.0,15.000000,-357.000000,0.601451,-0.467422,-1.300000,-2.000000,5.471125,0.59,-93.0,-64.0,-157.0,-2.0,74.0,75.0,149.0,-1.0,-0.047552,-41.0,-28.0,-69.0,-2.0,-25.0,5.0,-20.0,-3.0,-55.0,-30.0,-25.0,0.0,-0.090909,-6.0,-2.0,-8.0,-2.0,-23.809524,-37.0,-6.0,-43.0,-13.633952,-67.29,-119.493177,-0.936835,1.164284,-0.580863,-0.318041,-0.765198,-0.501005,-0.406280,0.394428,0.477757,-0.393652,-0.053924,-0.262823,-0.646837,-1.226970,-0.015496,-0.016648,-0.010952,-0.013096,-0.793182,1.820455,-4.261364,-1.802273,-1.190909,-0.897727,-0.840909,0.481818,0.065909,-0.777273,-0.078847,-2.459091,-0.575000,-2.922727,0.011268,0.013632,0.018430,0.048696,-2.0,-1.0,0.0,0.0,0.0,0.2820,-0.282991,1.041754,1.072008,1.048641,0.252306,-0.479154,-0.217931,0.632149,0.711967,0.023367,-0.328926,-0.070509,-0.073915,-0.968331,-0.009548,-0.020846,-0.002585,0.021385
10392,580,1.0,128,2.0,14982200,2014,1,13,105,5,2.0,104607,106000,1,0,195.000000,10.361944,4.0,0.053684,3.0,-181.000000,3908.000000,-4.747234,3.941824,6.672142,24.000000,-61.586636,-17.98,194.0,87.0,281.0,3.0,50.0,23.0,73.0,0.0,0.684932,17.0,10.0,27.0,0.0,138.0,33.0,171.0,2.0,135.0,98.0,37.0,1.0,-0.272059,14.0,4.0,18.0,3.0,-21.052632,35.0,13.0,48.0,72.916667,347.70,220.155039,3.904255,-0.723404,8.081560,8.074468,3.439716,-1.882979,0.322695,2.351064,3.014184,-3.882979,-0.153197,0.007092,-14.226950,0.074468,-0.075297,-0.04

## Baseline Model

In [61]:
baseline = df[['rank', 'Label']]
          
eval_df = baseline.copy()

y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

baseline_model = LogisticRegression()
print('Model: ', str(baseline_model))
training(baseline_model, X, y, SCALER=False, PCAer=False)
print('\n################################\n')

Model:  LogisticRegression()
Training time: 0:00:00.044923


Accuracy               : 65.586 %.
Precision              : 65.129 %.
Recall Score           : 66.943 %.
F1 Score               : 66.023 %.
Log-Loss               : 11.886 .
Brier Score            : 34.414 %.
Confusion Matrix
[[2796 1557]
 [1436 2908]]

Train Score            : 65.895 %.
Test Score             : 65.586 %.

################################



## Feature Selection

### Recursive Feature Elimination

In [64]:
eval_df = df.copy()
y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=20)
rfe = rfe.fit(X, y)
print('Selected features: %s' % list(X.columns[rfe.support_]))

Selected features: ['round', 'rank_points_log', 'spread', 'B365', 'matches_year_per', 'wins_tour', 'losses_tour', 'SvGms_career', 'svpt_year', '1stWon_year', '2ndWon_year', 'bpFaced_year', 'headtohead', 'Win_oppon', 'wstreak', 'lstreak', 'svpt_common_op', '1stWon_common_op', 'bpSaved_common_op', 'bpWon_common_op']


In [71]:
opt_ft1 = df[['round', 'rank_points_log', 'spread', 'B365', 'matches_year_per', 'wins_tour', 'losses_tour',
             'SvGms_career', 'svpt_year', '1stWon_year', '2ndWon_year', 'bpFaced_year', 'headtohead',
             'Win_oppon', 'wstreak', 'lstreak', 'svpt_common_op', '1stWon_common_op',
             'bpSaved_common_op', 'bpWon_common_op',
             'Label']]

eval_df = opt_ft1.copy()

y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

baseline_model = LogisticRegression()

print('Model: ', str(baseline_model))
training(baseline_model, X, y, SCALER=False, PCAer=False)
print('\n################################\n')

Model:  LogisticRegression()
Training time: 0:00:00.273382


Accuracy               : 75.451 %.
Precision              : 75.142 %.
Recall Score           : 75.99 %.
F1 Score               : 75.564 %.
Log-Loss               : 8.479 .
Brier Score            : 24.549 %.
Confusion Matrix
[[3261 1092]
 [1043 3301]]

Train Score            : 75.75 %.
Test Score             : 75.451 %.

################################



### Feature ranking with recursive feature elimination and cross-validation

In [67]:
eval_df = df.copy()
y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

rfecv = RFECV(estimator=LogisticRegression(), step=1, cv=5, scoring='accuracy')
rfecv.fit(X, y)

print("Optimal number of features: %d" % rfecv.n_features_)
print('Selected features: %s' % list(X.columns[rfecv.support_]))

Optimal number of features: 30
Selected features: ['round', 'rank_points_log', 'spread', 'B365', 'wins_career', 'matches_year_per', 'wins_tour', 'losses_tour', 'titles_tour', 'elo_rating', 'SvGms_career', '2ndIn_career', 'ace_year', 'df_year', 'svpt_year', '1stWon_year', '2ndWon_year', 'bpSaved_year', 'bpFaced_year', 'headtohead', 'Win_oppon', 'wstreak', 'lstreak', 'ace_common_op', 'svpt_common_op', '1stWon_common_op', 'SvGms_common_op', 'bpSaved_common_op', 'bpWon_common_op', '2ndSvrptWon_common_op']


In [72]:
opt_ft2 = df[['round', 'rank_points_log', 'spread', 'B365', 'wins_career', 'matches_year_per', 'wins_tour',
             'losses_tour', 'titles_tour', 'elo_rating', 'SvGms_career', '2ndIn_career', 'ace_year', 'df_year',
             'svpt_year', '1stWon_year', '2ndWon_year', 'bpSaved_year', 'bpFaced_year', 'headtohead', 'Win_oppon',
             'wstreak', 'lstreak', 'ace_common_op', 'svpt_common_op', '1stWon_common_op', 'SvGms_common_op',
             'bpSaved_common_op', 'bpWon_common_op', '2ndSvrptWon_common_op',
             'Label']]

eval_df = opt_ft2.copy()

y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

baseline_model = LogisticRegression()

print('Model: ', str(baseline_model))
training(baseline_model, X, y, SCALER=False, PCAer=False)
print('\n################################\n')

Model:  LogisticRegression()
Training time: 0:00:00.290429


Accuracy               : 77.877 %.
Precision              : 76.961 %.
Recall Score           : 79.512 %.
F1 Score               : 78.216 %.
Log-Loss               : 7.641 .
Brier Score            : 22.123 %.
Confusion Matrix
[[3319 1034]
 [ 890 3454]]

Train Score            : 77.574 %.
Test Score             : 77.877 %.

################################



### Manually Feature Selection

In [74]:
opt_ft3 = df[['elo_rating', 'Win_oppon', 'wins_career', 'rank_points',
          'matches_career', 'headtohead', 'spread', 'momentum',
          'losses_per_surface', 'home_advantage',
          'Label']]

eval_df = opt_ft3.copy()

y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

baseline_model = LogisticRegression()

print('Model: ', str(baseline_model))
training(baseline_model, X, y, SCALER=False, PCAer=False)
print('\n################################\n')

Model:  LogisticRegression()
Training time: 0:00:00.280256


Accuracy               : 78.636 %.
Precision              : 77.5 %.
Recall Score           : 80.64 %.
F1 Score               : 79.039 %.
Log-Loss               : 7.379 .
Brier Score            : 21.364 %.
Confusion Matrix
[[3336 1017]
 [ 841 3503]]

Train Score            : 78.342 %.
Test Score             : 78.636 %.

################################



## Hyperparameter Tuning

#### Scaling

In [75]:
eval_df = opt_ft3.copy()

y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

baseline_model = LogisticRegression()
print('Model: ', str(baseline_model))
training(baseline_model, X, y, SCALER=True, PCAer=False)
print('\n################################\n')

Model:  LogisticRegression()
USING SCALING..
Training time: 0:00:00.059557


Accuracy               : 78.579 %.
Precision              : 78.085 %.
Recall Score           : 79.397 %.
F1 Score               : 78.735 %.
Log-Loss               : 7.399 .
Brier Score            : 21.421 %.
Confusion Matrix
[[3385  968]
 [ 895 3449]]

Train Score            : 78.273 %.
Test Score             : 78.579 %.

################################



#### Hyperparameters

In [85]:
eval_df = opt_ft3.copy()

y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

param_grid = {'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag'],
              'penalty': ['l2', None],
              'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'max_iter': [100, 250]}

model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.783558 using {'C': 10, 'max_iter': 250, 'penalty': 'l2', 'solver': 'lbfgs'}


In [83]:
eval_df = opt_ft3.copy()
y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

param_grid = {'solver': ['liblinear'],
              'penalty': ['l1', 'l2'],
              'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'max_iter': [100, 250, 500]}

model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.783339 using {'C': 100, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


In [84]:
eval_df = opt_ft3.copy()
y = eval_df['Label']
eval_df.drop(columns=['Label'], inplace=True)
X = eval_df

param_grid = {'solver': ['saga'],
              'penalty': ['l1', 'l2', 'elasticnet', None],
              'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'max_iter': [100, 250, 500]}

model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.763367 using {'C': 0.001, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}
